# PetroKit — Technical Nodal Analysis Notebook

**Advanced, reproducible and auditable notebook** for nodal analysis, intended for engineering review, verification and integration into workflows.  

This notebook contains:  
- Rigorous explanation of the physics used in the Fase‑1 implementation.  
- Detailed parameter tables and unit conventions.  
- Verified code blocks that call `petrokit` modules (IPR, VLP, Flowline, Nodal).  
- Sensitivity studies, error handling and edge‑case checks.  
- Export of results and reproducible figures for reports.  

Use this notebook as a technical deliverable to accompany the PetroKit package (v0.1.0).

----
## Table of contents

1. [Assumptions and unit system](#assumptions)
2. [Physical background & equations](#physics)
3. [Quick sanity checks and helper wrappers]
4. [IPR validation and plots]
5. [VLP (tubing) validation and plots]
6. [Flowline (surface) validation and plots]
7. [Nodal analysis — intersection and robust root finding]
8. [Sensitivity studies & parametric sweeps]
9. [Export results & figures]
----


### 1. Assumptions and unit system {#assumptions}

**Unit convention used throughout this notebook** (documented explicitly to avoid errors):
- Pressure: **psi** (internal functions expect psi unless otherwise noted).  
- Volume: **STB/day** for production rates.  
- Length: **ft**.  
- Diameter: **in** (when passing to petrokit functions that expect inches).  
- Density: **lb/ft³**.  
- Viscosity: **cP**.

Key assumptions in Fase‑1 models:  
- Monophasic flow for VLP and flowline (no slip, no phase‑fraction models).  
- Darcy–Weisbach with constant friction factor `f` (default = 0.02).  
- Vogel IPR for solution‑gas drive reservoirs.  

Whenever a function returns values in SI, we convert explicitly back to engineering units for display.

### 2. Physical background & equations {#physics}

**IPR (Vogel)** — implemented form:
\[ q = q_{max} \left( 1 - 0.2 \frac{p_{wf}}{p_{res}} - 0.8 \left(\frac{p_{wf}}{p_{res}}\right)^2 \right) \]

**Fetkovich (linear)**:
\[ q = J \, (p_{res} - p_{wf}) \]

**Darcy–Weisbach (frictional pressure drop)**:
\[ \Delta p_{fric} = f \frac{L}{D} \frac{\rho v^2}{2} \]
where \(v = Q/A\), \(A=\pi D^2/4\), and we convert units as needed.

**Hydrostatic head**:
\[ \Delta p_{hydro} = \rho g H \]

In this technical notebook we include sanity checks, dimensional analysis and tests against theoretical limits.

In [ ]:
# Imports and plotting defaults
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
plt.rcParams.update({'figure.dpi': 120, 'font.size': 10})

# Try to import petrokit; if not available, inform the user
try:
    from petrokit import ipr, vlp, flowline, nodal, utils
    _PETROKIT_AVAILABLE = True
except Exception as e:
    _PETROKIT_AVAILABLE = False
    _IMPORT_ERROR = e
    print('Warning: petrokit not importable in this environment. Please ensure package is installed (pip install -e .)')
    print('Import error:', e)


#### 3. Quick sanity checks and helper wrappers

We include small wrappers that validate inputs before calling core functions. These are used throughout the notebook to avoid silent unit errors.

In [ ]:
def validate_positive(name, value):
    if value is None:
        raise ValueError(f'{name} is required')
    if isinstance(value, (list, np.ndarray)):
        if np.any(np.array(value) <= 0):
            raise ValueError(f'{name} must be positive (arrays)')
    else:
        if value <= 0:
            raise ValueError(f'{name} must be positive')

def safe_ipr_vogel(p_res, q_max, pwf):
    validate_positive('p_res', p_res)
    validate_positive('q_max', q_max)
    # bound pwf between 0 and p_res
    pwf = np.clip(pwf, 0, p_res)
    return ipr.vogel_ipr(p_res, q_max, pwf)

def safe_vlp_curve(q_range, well_depth, rho, mu, d, f=0.02):
    validate_positive('well_depth', well_depth)
    validate_positive('rho', rho)
    validate_positive('d', d)
    return vlp.vlp_curve(q_range, well_depth, rho, mu, d, f)


### 4. IPR validation and plots

We validate the Vogel IPR implementation at boundary conditions and plot the IPR curve.

In [ ]:
if _PETROKIT_AVAILABLE:
    # Parameters
    p_res = 3000.0
    q_max = 1200.0
    pwf_test = np.array([0.0, p_res/2, p_res])
    q_test = [safe_ipr_vogel(p_res, q_max, pwf) for pwf in pwf_test]
    print('Vogel IPR checks:')
    for pwf, qv in zip(pwf_test, q_test):
        print(f'  pwf={pwf:.1f} psi -> q={qv:.3f} STB/d')

    # Plot full curve
    pwf_arr, q_arr = ipr.ipr_curve_vogel(p_res, q_max, npts=200)
    plt.figure(figsize=(6,4))
    plt.plot(q_arr, pwf_arr, label='IPR (Vogel)')
    plt.scatter([q_max, q_arr[-1]], [0, pwf_arr[-1]], color=['black','red'])
    plt.xlabel('q [STB/d]')
    plt.ylabel('pwf [psi]')
    plt.title('Vogel IPR')
    plt.grid(True)
    plt.legend()
    plt.show()
else:
    print('Skipping IPR checks — petrokit not available')


### 5. VLP (tubing) validation and plots

We validate monotonicity and the hydrostatic limit.

In [ ]:
if _PETROKIT_AVAILABLE:
    q_range = np.linspace(10, q_max*1.2, 120)
    pwf_vlp = safe_vlp_curve(q_range, well_depth, rho, mu, d)
    # monotonic check
    monotonic = np.all(np.diff(pwf_vlp) > -1e-6)
    print('VLP monotonic (non-decreasing):', monotonic)
    # Hydrostatic approx at very low q
    pwf_low = safe_vlp_curve(np.array([1e-6]), well_depth, rho, mu, d)[0]
    p_hydro = (rho/144.0) * well_depth
    print(f'P_hydro (approx) = {p_hydro:.2f} psi; pwf_low = {pwf_low:.2f} psi')

    plt.figure(figsize=(6,4))
    plt.plot(q_range, pwf_vlp, label='VLP (Darcy-Weisbach)')
    plt.xlabel('q [STB/d]')
    plt.ylabel('pwf [psi]')
    plt.title('VLP (tubing)')
    plt.grid(True)
    plt.legend()
    plt.show()
else:
    print('Skipping VLP checks — petrokit not available')


### 6. Flowline (surface) validation and plots

We compute flowline ΔP vs q, and visualize the effect of diameter and elevation.

In [ ]:
if _PETROKIT_AVAILABLE:
    q_test = np.linspace(10, 5000, 80)
    dp = [flowline.flowline_pressure_drop(q, L=10000, d=6, rho=55, mu=1, elev=200) for q in q_test]
    plt.figure(figsize=(6,4))
    plt.plot(q_test, dp, label='ΔP flowline')
    plt.xlabel('q [STB/d]')
    plt.ylabel('ΔP [psi]')
    plt.title('Flowline ΔP vs q')
    plt.grid(True)
    plt.legend()
    plt.show()
else:
    print('Skipping flowline checks — petrokit not available')


### 7. Nodal analysis — robust intersection

Instead of relying on simple interpolation, we provide a robust root‑finding approach which minimizes |IPR(q) - VLP(q)| over q (direct in q space). This avoids interpolation artifacts when IPR and VLP are defined on different axes.

In [ ]:
if _PETROKIT_AVAILABLE:
    from scipy.optimize import minimize_scalar

    # Define objective: absolute difference between IPR(q) and q (from inverted VLP)
    def find_q_op(p_res, q_max, well_depth, rho, mu, d):
        qmin = 1.0
        qmax_search = max(1.2*q_max, 5000)
        def objective(q):
            # IPR gives q for a pwf; invert IPR numerically by computing pwf that gives q??
            # Simpler: compute pwf_vlp(q) and q_ipr(pwf_vlp).
            pwf_v = vlp.vlp_curve(np.array([q]), well_depth, rho, mu, d)[0]
            q_from_ipr = ipr.vogel_ipr(p_res, q_max, pwf_v)
            return abs(q - q_from_ipr)

        res = minimize_scalar(objective, bounds=(qmin, qmax_search), method='bounded')
        if not res.success:
            raise RuntimeError('Optimization failed to find q_op')
        return res.x, vlp.vlp_curve(np.array([res.x]), well_depth, rho, mu, d)[0]

    q_op_opt, pwf_op_opt = find_q_op(p_res, q_max, well_depth, rho, mu, d)
    print(f'Optimized Q_op = {q_op_opt:.2f} STB/d, pwf_op = {pwf_op_opt:.2f} psi')
    # Also show previous grid nodal for cross-check
    q_grid, pwf_grid = np.linspace(1, max(1.2*q_max,5000), 200), None
    q_ipr_grid = np.array([ipr.vogel_ipr(p_res, q_max, vlp.vlp_curve(np.array([q]), well_depth, rho, mu, d)[0]) for q in q_grid])
    pwf_vlp_grid = vlp.vlp_curve(q_grid, well_depth, rho, mu, d)

    plt.figure(figsize=(7,6))
    plt.plot(q_ipr_grid, np.linspace(0, p_res, len(q_ipr_grid)), label='IPR via mapping', color='blue')
    plt.plot(q_grid, pwf_vlp_grid, label='VLP', color='red')
    plt.scatter([q_op_opt], [pwf_op_opt], color='green', s=80, label='Optimized Q_op')
    plt.xlabel('q [STB/d]')
    plt.ylabel('pwf [psi]')
    plt.title('Nodal analysis (robust optimization)')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print('Skipping robust nodal analysis — petrokit not available or scipy missing')


### 8. Sensitivity studies & parametric sweeps

We run a set of parametric sweeps to illustrate the effect of:  
- Tubing diameter  
- Reservoir pressure `p_res`  
- Fluid density  

Results are organized into tabular outputs and summary plots.

In [ ]:
if _PETROKIT_AVAILABLE:
    diameters = [2.992, 3.5, 4.5, 6.0]  # in
    p_res_values = [2500, 3000, 3500]
    results = []
    for d_test in diameters:
        q_op_list = []
        for pres in p_res_values:
            try:
                qop, pwfop = find_q_op(pres, q_max, well_depth, rho, mu, d_test)
            except Exception:
                qop, pwfop = np.nan, np.nan
            q_op_list.append(qop)
        results.append(q_op_list)

    # Plot results
    plt.figure(figsize=(7,5))
    for idx, d_test in enumerate(diameters):
        plt.plot(p_res_values, results[idx], marker='o', label=f'D={d_test}\"')
    plt.xlabel('p_res [psi]')
    plt.ylabel('Q_op [STB/d]')
    plt.title('Sensitivity: Q_op vs p_res for different tubing diameters')
    plt.grid(True)
    plt.legend()
    plt.show()
else:
    print('Skipping sensitivity studies — petrokit not available')


### 9. Export results & figures

We store the summary table and export the main figure for use in reports.

In [ ]:
if _PETROKIT_AVAILABLE:
    out_dir = Path('output')
    out_dir.mkdir(exist_ok=True)
    import csv
    summary_csv = out_dir / 'nodal_summary.csv'
    with open(summary_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['diameter_in', 'p_res', 'q_op_stb_d', 'pwf_psi'])
        for i, d_test in enumerate(diameters):
            for j, pres in enumerate(p_res_values):
                writer.writerow([d_test, pres, results[i][j], ''])
    print('Wrote summary to', summary_csv)
    # Save last figure
    fig_path = out_dir / 'nodal_sensitivity.png'
    plt.savefig(fig_path, bbox_inches='tight')
    print('Saved figure to', fig_path)
else:
    print('Skipping export — petrokit not available')


----
## Appendix: References and further reading

- Vogel, J. R. (1968). "Electrostatic phenomena in petroleum production" (classic IPR reference).  
- Beggs, Hagedorn & Brown — correlations for multiphase flow in wells and pipelines.  
- White, F. M. — Fluid Mechanics (Darcy–Weisbach background).  

----
End of notebook.